In [1]:
import numpy as np

In [2]:
from pyomo.environ import *
from pyomo.environ import RangeSet
from pyomo.environ import value

In [3]:
A = np.array([
    [1,0,-1,0,0,0,0,0,0,0,0],
    [0,1,0,-1,0,0,0,0,0,0,0],
    [0,0,1,1,-1,0,0,0,0,0,0.6],
    [0,0,0,0,1,-1,0,0,0,0.5,0],
    [0,0,0,0,0,1,-1,0,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,-1,-1,0,0],
    [0,0,0,0,0,0,0,0,1,-1,-1]
])

In [11]:
np.shape(A)

(8, 11)

In [4]:
B = np.array ([
    [1,2,1,1,0,0,0,0.1,0.1,0.5,0.7]
])

In [5]:
C = np.array ([
    [1]
])

In [6]:
coef_GWP=C@B
coef_GWP=np.array(coef_GWP)
coef_GWP = coef_GWP.reshape(-1)

In [24]:
F=1
f=np.zeros(len(A))
f[5]=F

In [25]:
len(A)

8

In [31]:
positive_s_indices = []
positive_s_indices = list(range(1, len(np.transpose(A))+1))
#Model Formulation
# Create the model
model = ConcreteModel()

# Define the decision variable
model.set_s = RangeSet(len(np.transpose(A)))
model.s = Var(model.set_s)
model.set_positive_scale = Set(initialize=positive_s_indices)

#model constraint: As = f and s>=0
model.set_balance = RangeSet(len(f))
def balance(model, p): # As = f
    return sum(A[p-1,i-1]*model.s[i] for i in model.set_s) == f[p-1]
def positive_scale(model, i): # s>=0
    return (model.s[i]>=0)

model.balance_constraints = Constraint(model.set_balance, rule=balance)
model.positive_scale_constraints = Constraint(model.set_positive_scale, rule=positive_scale)

model.obj = Objective(expr = sum(coef_GWP[i-1]*model.s[i] for i in model.set_s), sense=minimize)
#Solver
solver = SolverFactory('gurobi')
solver.solve(model) # solves and updates instance

ERROR: Solver (gurobi) returned non-zero return code (1)
ERROR: Solver log: Set parameter Username Traceback (most recent call last):
      File "<stdin>", line 5, in <module> File
      "/Users/farshidnazemi/opt/anaconda3/envs/REMADE/lib/python3.9/site-
      packages/pyomo/solvers/plugins/solvers/GUROBI_RUN.py", line 67, in
      gurobi_run
        model = read(model_file)
      File "src/gurobipy/gurobi.pxi", line 3779, in gurobipy.read File
      "src/gurobipy/gurobi.pxi", line 112, in gurobipy.gurobi.read File
      "src/gurobipy/gurobi.pxi", line 60, in gurobipy.gurobi._getdefaultenv
      File "src/gurobipy/env.pxi", line 70, in gurobipy.Env.__init__
    gurobipy.GurobiError: Version number is 11.0, license is for version 10.0


ApplicationError: Solver (gurobi) did not exit normally

In [27]:
value(model.obj)

1.6

In [28]:
scaling_factors = []
for j in model.s:
    scaling_factors.append(model.s[j].value)       

In [29]:
scaling_factors

[0.5, 0.0, 0.5, 0.0, 0.5, 1.0, 1.0, -0.0, 1.0, 1.0, 0.0]